
 
#  **Introducción**

<font size = 4>
El presente notebook, recoge el procedimiento seguido para la creación del modelo y su posterior fase de entrenamiento, validación y test.  
<br> <br>
 
La base de datos utilizada para la creación del clasificador de imágenes se denomina **COVID-19 Radiography Database** y en su creación han participado un grupo de investigadores de la universidad de Qatar (Doha, Qatar), y de la universidad de Dhaka (Bangladesh).  
Se compone por **21.165 muestras**  distribuidas en **4 categorías** : **Covid-19, neumonía viral, opacidad torácica y normales**. 

</font>

#  **1 <span style="color:#0386f7de">|</span> Librerías** 

* <font size = 4>**Creación del modelo**: tensorflow, keras</font>
* <font size = 4>**Procesamiento de los datos**: numpy, sklearn, pandas, glob, shutil</font>
* <font size = 4>**Visualización de los datos**: matplotlib, cv2</font>


In [ ]:
#IMPORTS
import numpy as np 
import pandas as pd 
import os
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, losses
from tensorflow.keras.layers import BatchNormalization, Dropout, Conv2D, Flatten, Dense, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import glob
import shutil

imgSize = (299,299)
numClass = 3
batch_size = 64


#  **2 <span style="color:#0386f7de">|</span> Creacion de directorios temporales** 

<font size = 4>
    Es necesario realizar un tratamiento de los datos, sin embargo, Kaggle no permite editar lo datos de entrada.
    <br>
    Para solucionar este inconveniente se crean un directorio temporal y se copian los archivos para realizarles el tratamiento adecuado
</font>



In [ ]:
if(os.path.isdir('../output/images/') == False):
    os.makedirs('../output/images/')
if(os.path.isdir('../output/images/COVID') == False):
    os.makedirs('../output/images/COVID')
    print("Creado directorio COVID")
if(os.path.isdir('../output/images/Lung_Opacity') == False):
    os.makedirs('../output/images/Lung_Opacity')
    print("Creado directorio Lung_Opacity")
if(os.path.isdir('../output/images/Normal') == False):
    os.makedirs('../output/images/Normal')
    print("Creado directorio Normal")
if(os.path.isdir('../output/images/Viral_Pneumonia') == False):
    os.makedirs('../output/images/Viral_Pneumonia')
    print("Creado directorio Pneumonia")
 
path = glob.glob('/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/*/images/*.png')

for file in path:
    if((os.path.split(file)[1]).split('-')[0] == 'COVID'):
        dst = "../output/images/COVID/"
        if(os.path.isfile(dst + os.path.split(file)[1]) == False):
            shutil.copyfile(file, dst + os.path.split(file)[1])
    elif((os.path.split(file)[1]).split('-')[0] == 'Lung_Opacity'):
        dst = "../output/images/Lung_Opacity/"
        if(os.path.isfile(dst + os.path.split(file)[1]) == False):
            shutil.copyfile(file, dst + os.path.split(file)[1])
    elif((os.path.split(file)[1]).split('-')[0] == 'Normal'):
        dst = "../output/images/Normal/"
        if(os.path.isfile(dst + os.path.split(file)[1]) == False):
            shutil.copyfile(file, dst + os.path.split(file)[1])
    elif((os.path.split(file)[1]).split('-')[0] == 'Viral Pneumonia'):
        dst = "../output/images/Viral_Pneumonia/"
        if(os.path.isfile(dst + 'Viral_Pneumonia-' + (os.path.split(file)[1]).split('-')[1]) == False):
            shutil.copyfile(file, dst + os.path.split(file)[1])
            os.rename(dst + os.path.split(file)[1], dst + 'Viral_Pneumonia-' + (os.path.split(file)[1]).split('-')[1])
            
print("Todas las imagenes han sido almacenadas")

## **Método auxiliar para la comprobación de las rutas de las imagenes**

In [ ]:
glob.glob('../output/images/*/*.png')

# **3 <span style="color:#0386f7de">|</span> Creacion del dataframe y clasificacion de las imagenes** 

<font size = 4>Se crea una estructura de datos denominada DataFrame con la librería Panda donde se almacenan las rutas y la categoría de las imágenes.  </font>

In [ ]:
pathTemp = glob.glob('../output/images/*/*.png')
categories = []
filenames = []

dictCategory = {'COVID': '0', 'Lung_Opacity': '1', 'Normal': '2', 'Viral_Pneumonia': '3'}

for file in pathTemp:
    filenames.append(file)
    categories.append(dictCategory[(os.path.split(file)[1]).split('-')[0]])

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})
    
df.head()

# **4 <span style="color:#0386f7de">|</span> Creacion del conjunto de entrenamiento, validación y test** 
<font size = 4>Se ha procedido a la division de las imagenes para la creación de los conjuntos</font>
* <font size = 4>**Conjunto de entrenamiento**: 14.392 (68% del total)</font>
* <font size = 4>**Conjunto de validación**: 3.598 (17% del total)</font>
* <font size = 4>**Conjunto de test**: 3.175 (15% del total)</font>


In [ ]:
train_filenames, test_filenames = train_test_split(df, test_size=0.15)
train_filenames, val_filenames = train_test_split(train_filenames, test_size=0.2)


train_generator = ImageDataGenerator(
        rescale=1./255,
    )
test_generator = ImageDataGenerator(
        rescale=1./255,
    )

image_train = train_generator.flow_from_dataframe(
    train_filenames,
    target_size=(imgSize),
    x_col='filename',
    y_col='category',
    class_mode="categorical",
    shuffle=True,
    batch_size=batch_size,
)

image_val = train_generator.flow_from_dataframe(
    val_filenames,
    target_size=(imgSize),
    x_col='filename',
    y_col='category',
    class_mode="categorical",
    shuffle=True,
    batch_size=batch_size,
)

image_test = test_generator.flow_from_dataframe(
    test_filenames,
    target_size=(imgSize),
    x_col='filename',
    y_col='category',
    class_mode="categorical",
    shuffle=True,
    batch_size=batch_size,
)


## **Visualización de las imagenes**

In [ ]:
class_names = image_train.class_indices
classes = list(class_names.keys())

dictCategoryTr = {'0' : 'COVID', '1': 'Lung_Opacity', '2': 'Normal', '3': 'Viral_Pneumonia'}

images,labels = next(image_train)
labels = np.argmax(labels, axis=1)
class_dict = image_train.class_indices
class_dict_inv = dict((v, k) for k, v in class_dict.items())
y_names = [class_dict_inv[key] for key in labels]

plt.figure(figsize=(10, 10))
for image in images:
    j  = 0
    for i in range(4):
        ax = plt.subplot(1, 4, i+1)
        for k in range(len(labels)):
            if labels[k] == j:
                plt.imshow(images[k])
                plt.title(dictCategoryTr[classes[i]])
                plt.axis("off")
                break
Creacion del modelo        j = j+1

# **5 <span style="color:#0386f7de">|</span> Creacion del modelo** 

<font size = 4> </font>


<font size = 4>
    Se detallan las características del modelo generado 
     
</font> 
<font size = 4>
    Se ha implementado una arquitectura CNN que consta de 6 capas. Las primeras 5 capas están formadas por una capa convolucional (Conv2D), seguida por una capa de normalización (BatchNormalization), una capa de MaxPooling2D y una capa de Dropout. 
</font> 
<br>
<font size = 4>
    La ultima capa está formada por una capa de flatten y una capa densa 
</font>

In [ ]:
input_shape = (299, 299, 3)

myModel = keras.Sequential(
    [
        layers.Conv2D(128, kernel_size = (3, 3), activation="relu", padding='same', input_shape = input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.3),
        layers.Conv2D(128, kernel_size = (3, 3), activation="relu", padding='same', input_shape = input_shape),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Conv2D(64, kernel_size = (3, 3), activation="relu", padding='same'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Conv2D(64, kernel_size = (3, 3), activation="relu", padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.5),
        layers.Conv2D(32, kernel_size = (3, 3), activation="relu", padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(4, activation="softmax"),
    ]
   )

myModel.compile( optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy"])

myModel.summary()

# **6 <span style="color:#0386f7de">|</span> Entrenamiento** 
<font size = 4>Se realiza el entrenamiento del modelo con el conjunto de entrenamiento y sus respectivas etiquetas.</font>
   
    
<font size = 4> Se ha definido un *batch size* de 64 muestras y se ha fijado la duración del entrenamiento en un máximo de 75 épocas.</font>
<font size = 4> Además, se ha activado un mecanismo de EarlyStopping orientado a la pérdida del modelo.</font>

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', 
                               mode='min', 
                               patience = 5 ,
                               restore_best_weights=True)

model_chkpt = ModelCheckpoint('model_1.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

epoch = 75


history = myModel.fit(
    image_train,
    validation_data=image_val,
    batch_size=batch_size,
    epochs=epoch,
    callbacks=[model_chkpt ,early_stopping],
    verbose=0, 
)
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()))

print(max(history.history['accuracy']))
print(max(history.history['val_accuracy']))

<font size = 4>**Almacenamiento manual del modelo entrenado** </font>

In [ ]:
myModel.save('./model_1.h5')

<font size = 4>**Carga del modelo entrenado** </font>

In [ ]:
myModel = keras.models.load_model('../input/tfg-model/model_1.h5')

# **7 <span style="color:#0386f7de">|</span> Validación**

<font size = 4>Se utiliza el método evaluate disponible en la librería Keras para evaluar el rendimiento del modelo entrenado y comparar los resultados con los obtenidos previamente.</font>

In [ ]:
# printing model accuracy for train and test data

train_evaluation = myModel.evaluate(image_train)
print(f"Train Accuracy: {train_evaluation[1] * 100:.2f}%\n")

test_evaluation = myModel.evaluate(image_val)
print(f"Test Accuracy : {test_evaluation[1] * 100:.2f}%")

# **8 <span style="color:#0386f7de">|</span> Predicción**
<font size = 4>Para finalizar, se ha realizado una última comprobación con la función predict perteneciente a la librería Keras. Esta función predecirá la clasificación de las imágenes que se introduzcan como entrada en el modelo. </font>

In [ ]:
class_namesT = image_test.class_indices
print(class_namesT)
classesT = list(class_namesT.keys())

imagesT,labelsT = next(image_test)
labelsT = np.argmax(labelsT, axis=1)
class_dictT = image_test.class_indices
class_dict_invT = dict((v, k) for k, v in class_dictT.items())
y_names = [class_dict_invT[key] for key in labelsT]

plt.figure(figsize=(20,20))
dictCatTr = {'0' : 'COVID', '1': 'Lung_Opacity', '2': 'Normal', '3': 'Viral_Pneumonia'}
ls = next(image_test)
_, labelsT = ls
labelsT = np.argmax(labelsT, axis=1)
prediction = myModel.predict(ls[0])

for i in range(16):
    
    pred = np.argmax(prediction[i])
    
    plt.subplot(4, 4, i+1)
    plt.imshow(ls[0][i])
    plt.title(f'Real:{dictCatTr[str(labelsT[i])]} \n Predicción:{dictCatTr[str(pred)]}')
    plt.axis("off")